# Lecture 02 : GNN & `torch_geometric`

written by SinsuSquid (bgkang) on 26 November 2024

이제야 들꽃반이 만들어진 이유에 조금 더 가까운 주제로 들어섰네요. 이번 시간에는 그래프에 대한 얘기를 주구장창 하게 될 것 같은데, 일단 알아두어야 할 것은 이 '그래프'는 우리가 논문에서 맨날 보는 그런 2차원 형태의 '그래프'가 아니라 자료구조 (data structure) 분야에서의 '그래프'를 의미합니다. 걱정 마요, '그래프'가 무엇인지부터 시작할거니까. 어쨌든 이런 '그래프'의 개념이 포함된 인공신경망이니까 Graph Neural Network (GNN)이라 하는거겠죠? PyTorch에서 Graph를 사용하기 위한 라이브러리에는 크게 `dgl` (Deep Graph Library)와 `torch_geometric` (PyG) 두가지 종류가 있어요. 솔직히 `torch_geometric`만 써봤기때문에 뭐가 더 좋은건진 잘 모르겠어요. 논문 읽어봤을때는 둘 다 비슷한 빈도로 쓰이긴 하는거 같긴 하더라고요. 어쨌든 이 수업은 전지적 강의자 시점으로 진행되고 있으니까 우리는 `torch_geometric`을 이용해서 Graph 형태의 data를 다루어보도록 하겠습니다.

## What is a "Graph"?

![A Graph](https://www.simplilearn.com/ice9/free_resources_article_thumb/Graph%20Data%20Structure%20-%20Soni/what-is-graphs-in-data-structure.png "simpleilearn.com")

아까 말한거처럼 지금부터는 우리가 흔히 알던 그래프는 잠시 잊어두도록 하죠. Graph란 node (or vertices)와 edge로 구성되어 있는 자료 구조를 의미하는데, 여기서 edge는 node와 node 사이의 '연결'을 나타내기 때문에 Graph는 이 '연결 (또는 관계)'를 표현하기에 적합한 형태입니다. 수식으로 한번 나타내보자면 Graph ($G$)는 $G = \{V, E\}$로 표현할 수 있고 여기서 $V$와 $E$는 각각 node와 edge의 set을 의미합니다. Node는 $n_i \in V$가 되겠고, edge는 다음과 같이 표현해볼까요? Node $n_k$ 에서 node $n_l$로 가는 edge를 $e_j = (n_k \rightarrow n_l) \in E$와 같이 나타낼 수 있을겁니다. $(n_k \rightarrow n_l)$와 $(n_l \rightarrow n_k)$를 구분하지 않는 경우를 undirected라 하고, 구분하는 경우는 directed graph라 해요.


잠시 잊고 있었을지도 모르겠지만 우리는 '화학과'이기 때문에 화학스러운 비유를 한번 들어볼게요. 만약 node를 하나의 atom으로 생각한다면, edge는 다시말해 atom과 atom 사이의 연결이니까 chemical bonding에 해당하겠죠? 추가로 우리가 C-H 와 H-C bond를 굳이 구분하진 않으니까 chemical compound의 경우 undirected graph 형태로 표현하는것이 바람직하겠네요.

## How to represent a graph in PyTorch?

우리가 직면한 다음 문제는 어떻게 Graph를 컴퓨터님께서 알아먹을 수 있게 하는가입니다. 모니터 위에 슥슥 그림을 그렸을 때 알아서 해주면 좋겠지만 좀 그렇잖아요. 여기서 `torch_geometric`의 역할은 graph 구조를 PyTorch상에서 구현할 수 있도록 하는 '표준'을 정해준다고 생각하면 될 것 같아요. 아까도 말했듯이 graph의 핵심은 바로 '연결'입니다. '연결할 대상 (node)'와 '연결 (edge)'를 정의한다면 그래프를 표현할 수 있죠. 역시 예시만큼 좋은게 없을 것 같습니다.

![A model Graph](https://pytorch-geometric.readthedocs.io/en/latest/_images/graph.svg "pytorch-geomtric.readthedocs.io")

`torch-geometric`에서 위 graph는 다음과 같이 표현됩니다.

In [ ]:
import torch
from torch_geometric.data import Data # 하나의 Data object가 하나의 graph!

x = torch.tensor(
    [[-1],
     [0],
     [1]], dtype = torch.long
)

edge_index = torch.tensor(
    [[0, 1, 1, 2],
     [1, 0, 2, 1]], dtype = torch.float
)

graph = Data(x = x, edge_index = edge_index)

graph

주어진 예시를 바탕으로 하나씩 뜯어보도록 하죠.

- `data.x` : node를 표현하는 부분이겠죠
  - $N$개의 node가 존재한다고 하면 `data.x`의 shape는 $(N, D_{\rm node})$입니다
  - $D_{\rm node}$는 node 하나가 갖고있는 data의 dimension이고, 이 예시에서 $D_{\rm node} = 1$이겠네요

- `data.edge_index` : edge를 표현하는 부분이에요
  - $M$개의 edge가 존재할 때 `data.edge_index`의 shape는 $(2, M)$입니다
  - "뭐야 이거 어떻게 읽어"라고 생각할 수 있을 것 같은데, `edge_index[0]`을 출발 node, `edge_index[1]`을 도착 node라고 생각해보세요
  - `edge_index = [[0, 1], [1, 0]]`을 생각해보면 $(n_0 \rightarrow n_1)$와 $(n_1 \rightarrow n_0)$를 구분하고 있는 것 같죠?
  - 맞아요, `torch_geometric`에서는 undirected graph를 크게 신경쓰지 않아요 (왜 그런지는 나중가면 알게될거에요)

지금까지는 node에만 information이 담겨있는 가장 단순한 형태의 graph를 표현하는 방법을 알아보았어요. 하지만 가끔은 edge에도 info를 집어넣을 수 있잖아요? 예시를 한번 들어보자면, 우리가 사람 하나하나를 node로 표현하는 그래프를 그린다고 할 때, 사람 사이의 호감도를 edge에 담아 표현할 수 있겠죠?

![NewJeans Love](./Figures/NewJeans.png)

*숫자는 현실을 반영하지 않았음을 밝힙니다.*

이처럼 edge에도 data를 넣고싶다면 이용하는것이 바로 `data.edge_attr`입니다.

In [ ]:
# x, edge_index, edge_attr의 type을 신경써보도록 해요!
# gpu를 사용하거나 다른 CPU architecture를 사용하는 경우 다른 설정을 써야할수도 있어요

x = torch.tensor([[0],                      # 김민지
                  [1],                      # 강해린
                  [2],                      # 이혜인
                  [3],                      # 팜하니
                  [4]], dtype = torch.float) # 모다니

# 변화구를 좀 줘봤죠?
edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [0, 2],
                           [2, 0],
                           [0, 3],
                           [3, 0],
                           [0, 4],
                           [4, 0],
                           [1, 2],
                           [2, 1],
                           [1, 3],
                           [3, 1],
                           [1, 4],
                           [4, 1],
                           [2, 3],
                           [3, 2],
                           [2, 4],
                           [4, 2],
                           [3, 4],
                           [4, 3]], dtype = torch.long)

# 엉엉엉 내가 왜 이런 노가다를 자처했을까 엉엉엉
edge_attr = torch.tensor([[7], [5], [1], [11], [42], [1], [11], [11], [5], [5], 
                          [7], [5], [42], [11], [3], [11], [5], [5], [7], [5]], dtype = torch.long)

newJeans = Data(x = x, edge_index = edge_index.t().contiguous(), edge_attr = edge_attr)
newJeans

만약 graph가 $M$개의 edge를 가지고있다면 edge_attr은 $(M, D_{\rm edge})$의 shape를 갖게 되겠고, $D_{\rm edge}$는 역시 edge 하나가 갖는 info의 dimension이 되겠지요.

## Message Passing

[출처](https://process-mining.tistory.com/164) (고마워요!)

"왜 GNN을 쓰냐?"라고 물어본다면 저는 Message Passing (MP)라고 답하겠어요. MP란 node의 neighbor 정보를 이용해 해당 node의 상태를 update하는것을 의미한다고 하네요. MP는 'aggregation'과 'update' 두 단계로 구성되어 있는데 이를 '이웃으로부터 정보를 모으는 단계'와 '모은 정보를 바탕으로 내 정보를 update하는 단계'로 생각할 수 있겠습니다. 좀 더 컴돌이같이 말해보면 'aggregation으로부터 message를 만드는 단계'와 'current state와 message를 이용해 state를 update하는 단계'라고 할 수 있습니다.

![An Example Graph](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FSBych%2Fbtrx5z4ecgk%2FNVlNuWOvgaVcC7sA5eulN0%2Fimg.png "process-mining.tistory.com")

![Aggregation](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FdjBgUE%2Fbtrx6oIak2x%2FKjYOB3uVr9q9KZgtKxTKqK%2Fimg.png "process-mining.tistoty.com")

- aggregation
  - $m^{t+1}_v = \sum_{w \in N(v)} M_t(h^t_v, h^t_w, e_{vw})$라 수식으로 표현한다는군요
  - $h^t_v$ : $t$ 단계에서 node $v$에 저장되어있는 정보
  - $w$ : node $v$의 neighboring node
  - $e_{vw}$ : node $v$와 node $w$사이 edge에 저장되어있는 정보
  - $m^{t+1}_v$ : $t+1$ 단계에서 생성되는 message 정보
  - $M_t()$ : message function, 다시말해 $h^t_v, h^t_w, e_{vw}$를 어떻게든 지지고볶아서 출력값을 내는 무언가
  - ('정보' 라고 써놓은건 다 vector에 해당한다고 생각하면 됩니다)

- update
  - $h^{t+1}_v = U_t(h^t_v, m^{t+1}_v)$와 같이 표현한답니다
  - $U_t()$ : update function, 다시말해 $h^t_v, m^{t+1}_v$를 어떻게든 지지고볶아서 출력값을 내는 무언가

여기서 '지지고볶는'과정을 어떻게 달리 하느냐가 해당 방법을 상징할 수 있는 특징이겠죠. 이런 detail들에 대해서는 지금 당장 알 필요는 없을 것 같고, 나중에 이야기해보도록 하죠.

그림에서도 나타나있지만, node A의 update에 사용될 message를 얻는 과정에 대해 생각해볼게요. message passing 과정을 모든 node에 대해 여러번 반복 (iteration) 한다고 하면, A의 message를 만들기 위해 B, C, D node의 정보를 필요로 하게 될것이고 이 B, C, D의 update를 위해서는 각각의 neighbor로부터 aggregation이 더 필요하겠죠. 그래서 충분히 많은 수의 iteration이 주어진다면 graph 내부에서 node가 갖고 있는 정보는 더 이상 변화하지 않고 '수렴'하게 될거에요 (양자 계산의 self-consistency를 생각해보면 비유가 되려나요?). 우리는 이처럼 '수렴' 되어있는 graph를 가지고 이런저런 일들을 할 수 있을거에요.

## Wait, wait a second.

여기까지 우리는 graph으로 표현되는 자료구조와 이 사이에서 node간 message 교환이 어떻게 일어나는지에 대해 알아보았어요. 그런데 가장 큰 문제가 하나 남아있죠. "아니 이걸 지금 나보고 짜오라고? 미친거 아냐?" 걱정 마세요. 아무도 당신에게 그런 버거운 부탁 하지 않았어요. 이게 바로 우리가 `torch_geometric`을 사용하는 이유죠. `torch_geometric`에는 여러 message passing neural network (MPNN)의 layer가 이미 정의되어있고, 우리는 이걸 가져다 쓰기만 하면 되는겁니다! 이번 시간에는 딱히 명확한 target을 설정한게 아니었으니까, 아까 정의해봤던 graph를 가지고 대충 한번 만들어보기로 할게요.

In [ ]:
from torch_geometric.nn.conv import GATConv
# GATConv는 Graph Attention Network의 약자로, MPNN 종류중 하나에요.

class simpleMPNN(torch.nn.Module):
    def __init__(self, in_channels, out_channels, edge_dim):
        super().__init__()
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.edge_dim = edge_dim
        
        self.gat = GATConv(in_channels = self.in_channels, # node vector dimension
                           out_channels = self.out_channels, # output vector dimension
                           edge_dim = self.edge_dim, # edge vector dimension
                          )

        self.reset_parameters()

        return

    def forward(self, x, edge_index, edge_attr):
        return self.gat(x = x,
                        edge_index = edge_index,
                        edge_attr = edge_attr,
                       )

    def reset_parameters(self):
        self.gat.reset_parameters()

model = simpleMPNN(in_channels = 1, out_channels = 5, edge_dim = 1)
model

In [ ]:
# before MPNN
newJeans.x

In [ ]:
# after MPNN
model(x = newJeans.x,
      edge_index = newJeans.edge_index,
      edge_attr = newJeans.edge_attr)

짜잔, 어떤 알고리즘인지는 모르겠지만 주어진 input으로부터 MPNN을 이용해 node 정보를 실제로 update해볼수 있었어요. 물론 지금 당장은 그닥 쓸모 없는 값들이지만 조만간 큰 일을 도모할 수 있도록 발전시켜야겠죠. 

이처럼 `torch_geometric`에서 제공하는 layer를 잘 활용하기 위해서 [conv.GATConv](https://pytorch-geometric.readthedocs.io/en/stable/generated/torch_geometric.nn.conv.GATConv.html#torch_geometric.nn.conv.GATConv)와 같은 페이지를 잘 활용해야 해요. Layer의 특징은 무엇인지, layer object를 initialize하기 위해 어떤 input들이 사용되는지, 또 forward 함수에는 어떤 parameter를 필요로 하는지 공부할 수 있죠.

추가로, '어떤 layer를 사용할 수 있을까?'에 대한 걱정은 [GNN Cheatsheet](https://pytorch-geometric.readthedocs.io/en/stable/cheatsheet/gnn_cheatsheet.html)같은 페이지가 도움이 될 수 있을거에요. 각 layer들이 어떤 종류의 input을 지원하는지 표로 정리되어있고, 심지어 각 layer가 소개된 논문 또한 인용해주고 있어요. (Paper가 활발하게 인용된다는 건 이 분야가 계속해서 develop되고있다는 증거이기도 해요!) 물론 저도 이 모든 layer들이 어떤 역할을 하는지, 특징이 무엇인지 하는 정보들은 알 수 없어요. 반대로 이건 앞으로 여러분들이 더 공부해야 한다는걸 의미하기도 하죠!

## OK, what can GNNs do?

이번 Lecture의 마지막은 과연 GNN이 어떤 일을 할 수 있는지에 대해 간단하게만 이야기해보며 마치도록 하죠. 눈치 빠른 친구들은 알겠지만 지금까지 우리가 한거라고는 고작 하나의 graph에서 node정보를 update하는게 다였어요. 사실 GNN의 무궁무진한 활용은 지금부터 시작이 될거에요. 저도 [Chemomile](https://github.com/SinsuSquid/Chemomile) (따... 딱히 조회수를 올려달라는건 아니니까...!)을 준비했지만, GNN으로 가능한 일중 하나의 단편만을 보여준게 아닌가 싶어요. "GNN으로 이런게 가능하다!!!"라는걸 보여줄 수 있는 insight만 던져주고 끝낼 생각이니, 앞으로의 발전은 여러분께 기대해보겠어요.

- Chemomile의 경우와 비슷한데, 하나의 graph가 하나의 target을 갖는다고 하면, graph 안의 node 정보를 MPNN을 이용해서 update하고, 이 update된 node 정보들을 종합해서 하나의 target에 연결시키는 model을 구성할 수 있겠죠.
- 이번엔 node마다 target을 가지고있다고 합시다. MPNN을 이용해 graph 내부의 node 정보를 update하고, 이번에는 node각각의 정보를 node 각각의 target에 연결시키는 model을 만들수도 있을거에요.
- protein 연구하시는분들이 자주 사용하는 방법인데, 2개의 protein을 각각의 graph로 나타낸다고 해보죠. (나도 몰라요 저한테 물어보지 마세요) 그 다음에는 각 graph에서 얻어진 정보를 가지고 2개의 protein이 docking을 하는지 안하는지 판별하는 model을 만들수도 있습니다!

## Adiós,

그동안은 이름만 'GNN 들꽃반'이었지 그래프는 꺼내지도 않았는데, 이제 와서야 그래프와 관련된 이야기를 조금 더 꺼낼 수 있게 되었네요. 여러분이 GNN을 직접 건드릴 일이 있다면 이번 Lecture에서 나왔던 graph를 정의하고, graph를 layer에 어떻게 전달하는지에 대한 과정들이 많이 필요할거라고 확신할 수 있어요. 그만큼 이부분은 coding도 coding이지만, 일단 머리로 이해를 잘 해본다음 손으로 옮겨보는 연습이 많이 필요할 것 같아요. 노오오력은 우리 들꽃반 학생들에게 기대해보기로 하고, 저는 이만 퇴장해보도록 할게요. 안녕!